<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Evaluate_Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this notebook we will evaluate the results of the previously generated summaries with different both extractive and abstractive models. For this we will use both manually implemented and other methods like BLANC

In [1]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)
!pip install -U sentence-transformers
!pip install blanc

Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (507/507), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 507 (delta 242), reused 476 (delta 229), pack-reused 0
Receiving objects: 100% (507/507), 29.39 MiB | 15.33 MiB/s, done.
Resolving deltas: 100% (242/242), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b26d7a5c995cbf43e262ca28f6702156e98f26a6703a05d5c9644300e8cb1310
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built

In [2]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
df = pd.read_csv('/content/summaries.csv')
df.head()

,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,Welcome back to Pattern Recognition.\nToday we...,Support vector machines are a very powerful cl...,Support vector machines are a powerful classif...,Today we want to start talking about support v...,Today we want to start talking about support v...
1,Welcome back to pattern recognition.\nToday we...,Today we want to look into a couple of the adv...,Learn how to introduce the kernel PCA.\nLook i...,Today we want to look into a couple of the adv...,Today we want to look into a couple of the adv...
2,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,NaN,NaN
3,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...,Today we want to look a bit more into convex o...,Today we want to look a bit more into convex o...
4,"In 2013, Grahame Greive, the leader of the gl...","The Clinical Information Modeling Initiative, ...",There are many strongly held points of view ab...,"In 2013, Grahame Greive, the leader of the glo...","In 2013, Grahame Greive, the leader of the glo..."


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
def cosine_similarity(a, b):
    nominator = np.dot(a, b)
    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    
    denominator = a_norm * b_norm
    
    cosine_similarity = nominator / denominator
    
    return cosine_similarity

def evaluate(model, paragraph, summary):
  # Get sentence embedding for paragraph
  paragraph_embeddings = model.encode(sent_tokenize(paragraph))

  # Get sentence embedding for summary
  summary_embeddings = model.encode(sent_tokenize(summary))

  result = np.array([])
  # loop through summary sentences and find arg max cosine similarity
  for summary in summary_embeddings:
    max = 0
    for paragraph in paragraph_embeddings:
      similarity = cosine_similarity(summary, paragraph) 
      if  similarity > max:
        max = similarity
    result = np.append(result, max)
  # return the mean
  return np.mean(result)

In [6]:
from blanc import BlancHelp, BlancTune

# blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=False)
blanc_help = BlancHelp(device='cuda')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
metrics = pd.DataFrame()

### Abstractive

In [8]:
metrics['bart_score'] = df[['content', 'bart']].apply(lambda x: evaluate(model, x['content'], x['bart']), axis=1)
metrics['pegasus_score'] = df[['content', 'pegasus']].apply(lambda x: evaluate(model, x['content'], x['pegasus']), axis=1)

In [9]:
metrics['pegasus_blanc'] = df[['content', 'pegasus']].apply(lambda x: blanc_help.eval_once(x['content'], x['pegasus']), axis=1)
metrics['bart_blanc'] = df[['content', 'bart']].apply(lambda x: blanc_help.eval_once(x['content'], x['bart']), axis=1)

100%|██████████| 372/372 [00:08<00:00, 41.64it/s]


In [10]:
df.dropna(inplace=True)

In [11]:
metrics

,bart_score,pegasus_score,pegasus_blanc,bart_blanc
0,0.892160,0.847748,0.124772,0.148452
1,0.880205,0.815136,0.086192,0.195816
2,0.380143,0.377958,0.322581,0.354839
3,0.937891,0.889664,0.166667,0.227488
4,0.742501,0.911712,0.137592,0.100737
5,0.828703,0.855091,0.121739,0.110870
6,0.965930,0.810819,0.077465,0.112676
7,0.908323,0.868556,0.146038,0.115484
8,0.902592,0.801659,0.118089,0.180887
9,0.811754,0.908299,0.132812,0.164931


### Extractive

In [12]:
metrics['bert_extractive_blanc'] = df[['content', 'bert_extractive']].apply(lambda x: blanc_help.eval_once( x['content'], x['bert_extractive']), axis=1)
metrics['dist_bert_extractive_blanc'] = df[['content', 'dist_bert_extractive']].apply(lambda x: blanc_help.eval_once( x['content'], x['dist_bert_extractive']), axis=1)

100%|██████████| 372/372 [00:16<00:00, 22.84it/s]


In [ ]:
# metrics['bert_extractive_score'] = df[['content', 'bert_extractive']].apply(lambda x: evaluate(model, x['content'], x['bert_extractive']), axis=1)
# metrics['dist_bert_extractive_score'] = df[['content', 'dist_bert_extractive']].apply(lambda x: evaluate(model, x['content'], x['dist_bert_extractive']), axis=1)

In [13]:
metrics.head()

,bart_score,pegasus_score,pegasus_blanc,bart_blanc,bert_extractive_blanc,dist_bert_extractive_blanc
0,0.892160,0.847748,0.124772,0.148452,0.235883,0.289617
1,0.880205,0.815136,0.086192,0.195816,0.276987,0.313808
2,0.380143,0.377958,0.322581,0.354839,NaN,NaN
3,0.937891,0.889664,0.166667,0.227488,0.311216,0.300158
4,0.742501,0.911712,0.137592,0.100737,0.228501,0.262899


In [14]:
metrics.describe()

,bart_score,pegasus_score,pegasus_blanc,bart_blanc,bert_extractive_blanc,dist_bert_extractive_blanc
count,13.000000,13.000000,13.000000,13.000000,12.000000,12.000000
mean,0.825271,0.803675,0.136418,0.164658,0.247697,0.262661
std,0.146658,0.135754,0.061961,0.068539,0.047108,0.050757
min,0.380143,0.377958,0.077465,0.100737,0.165493,0.169014
25%,0.820286,0.793534,0.113924,0.115484,0.222127,0.244289
50%,0.832919,0.815136,0.124772,0.148452,0.246542,0.261367
75%,0.902592,0.868556,0.145051,0.180887,0.280266,0.295176
max,0.965930,0.911712,0.322581,0.354839,0.319444,0.345486


In [15]:
print('BART score ', metrics['bart_score'].mean())
print('Pegasus score', metrics['pegasus_score'].mean())

BART score  0.8252713528702917
Pegasus score 0.8036747200091382


In [16]:
%cd ..
metrics.to_csv('results_1.csv', index=False)
metrics.describe().to_csv('results_stats.csv', index=False)

/content


In [19]:
metrics

,bart_score,pegasus_score,pegasus_blanc,bart_blanc,bert_extractive_blanc,dist_bert_extractive_blanc
0,0.892160,0.847748,0.124772,0.148452,0.235883,0.289617
1,0.880205,0.815136,0.086192,0.195816,0.276987,0.313808
2,0.380143,0.377958,0.322581,0.354839,NaN,NaN
3,0.937891,0.889664,0.166667,0.227488,0.311216,0.300158
4,0.742501,0.911712,0.137592,0.100737,0.228501,0.262899
5,0.828703,0.855091,0.121739,0.110870,0.191304,0.252174
6,0.965930,0.810819,0.077465,0.112676,0.165493,0.169014
7,0.908323,0.868556,0.146038,0.115484,0.203004,0.189539
8,0.902592,0.801659,0.118089,0.180887,0.257338,0.225256
9,0.811754,0.908299,0.132812,0.164931,0.319444,0.345486


In [22]:
# load t5 results and test
df_t5 = pd.read_csv('/content/summaries_t5.csv')
df_t5

,content,t5
0,Welcome back to Pattern Recognition!\nToday we...,we have a set of new vectors that are lambda a...
1,Welcome back to Pattern Recognition!\nToday we...,the laplacian support vector machine is essent...
2,Welcome back to pattern recognition.\nToday we...,this is the longest projection of our negative...
3,Welcome back to pattern recognition.\nToday we...,we want to introduce the kernel PCA and look i...
4,"The Chrysler Building, the famous art deco New...",the building sold fairly quickly after being p...
5,The most sophisticated form of HIE creates sem...,the most sophisticated form of HIE creates sem...
6,Welcome back to pattern recognition.\nToday we...,we want to talk about optimization and of cour...
7,dr karen bell is the chair of the certificatio...,dr karen bell is the president of the national...
8,Dr. Karen Bell is the chair of the Certificat...,health informatics is a new era of information...
9,"All right, so here is a rule we're going to c...",if we repeat this update rule over the finite ...


In [26]:
# test the results of t5
metrics_t5 = pd.DataFrame()
metrics_t5['t5_score'] = df_t5[['content', 't5']].apply(lambda x: evaluate(model, x['content'], x['t5']), axis=1)
metrics_t5['t5_blanc'] = df_t5[['content', 't5']].apply(lambda x: blanc_help.eval_once(x['content'], x['t5']), axis=1)

100%|██████████| 400/400 [00:04<00:00, 81.82it/s]


In [27]:
metrics_t5.to_csv('results_t5_1.csv', index=False)
metrics_t5.describe().to_csv('results_stats_t5.csv', index=False)

# New Section

## While running code part below RAM is filled thus restart the notebook

In [ ]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd.read_csv('/content/summaries.csv')
df.head()

,Unnamed: 0,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,0,dr karen bell is the chair of the certificatio...,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,NaN,NaN
1,1,Welcome back to Pattern Recognition!\nToday we...,Today we look at the concept of duality in con...,duality is generally a concept that you can in...,Today we want to look a bit more into convex o...,Today we want to look a bit more into convex o...
2,2,Welcome back to Pattern Recognition!\nToday we...,Today's topic is the Laplacian Support Vector ...,The Laplacian Support Vector Machine allows yo...,Today we want to talk a bit about the Laplacia...,Today we want to talk a bit about the Laplacia...
3,3,Welcome back to pattern recognition.\nToday we...,Today we want to look a bit more into optimiza...,We've seen the gradient descent methods in the...,Today we want to look a bit more into optimiza...,Today we want to look a bit more into optimiza...
4,4,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,"The Chrysler Building, the famous art deco New...","The Chrysler Building, the famous art deco New..."


In [18]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 8.0 MB/s eta 0:00:00


In [17]:
from bert_score import score

ModuleNotFoundError: ignored

In [ ]:
def evaluate_bert_score(paragraph, summary):
  # Get sentence embedding for paragraph
  paragraph_sentences = sent_tokenize(paragraph)
  
  # Get sentence embedding for summary
  summary_sentences =sent_tokenize(summary)

  result = np.array([])
  # loop through summary sentences and find arg max cosine similarity
  for summary in summary_sentences:
    max = 0
    for paragraph in paragraph_sentences:
      P, R, F1 = score([summary], [paragraph], lang="en", verbose=True)
      similarity = F1.mean() 
      if  similarity > max:
        max = similarity
    result = np.append(result, max)
  # return the mean
  return np.mean(result)

In [ ]:
metrics['bart_bertScore'] = df[['content', 'bart']].apply(lambda x: evaluate_bert_score(x['content'], x['bart']), axis=1)
metrics['pegasus_bertScore'] = df[['content', 'pegasus']].apply(lambda x: evaluate_bert_score(x['content'], x['pegasus']), axis=1)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.14 seconds, 7.22 sentences/sec


In [ ]:
metrics.head()

NameError: ignored

In [ ]:
metrics.describe()

In [ ]:
%cd ..
metrics.to_csv('results_1.csv', index=False)

In [ ]:
metrics.to_csv('results_2.csv', index=False)

In [ ]:
metrics.describe().to_csv('results_described.csv', index=False)